In [164]:
from fuzzywuzzy import fuzz
import re
import pandas as pd

# load the data frame
input_path = "../data/music_picks/model_statistics.csv"
df = pd.read_csv(input_path)

# rename artist and title column to artist_old and title_old
df.rename(columns={"artist": "artist_old"}, inplace=True)
df.rename(columns={"title": "title_old"}, inplace=True)

def string_cleaner(string):
    # remove unwanted characters from string
    unwanted_characters = [',', '_', '\n']
    string = re.sub('|'.join(unwanted_characters), '', string)
    # remove numbers within parentheses from string
    string = re.sub(r'\([^)]*\)', '', string)
    # remove numbers directly after a dot from string
    string = re.sub(r'\.\d+', '', string)
    # insert spaces between words of a string written in camel case
    string = re.sub(r'(?<!^)(?=[A-Z])', ' ', string)
    return string

# compare strings in the list sequentially and return a list of the same size with the replaces values
def find_and_replace_duplicates(str_list, threshold=75):
    # clean the list of strings
    str_list = [string_cleaner(string) for string in str_list]
    # create a list with the same size as the input list
    similar_strs = [None] * len(str_list)
    for i, s in enumerate(str_list):
        if similar_strs[i] is None and not ',' in s and not any([fuzz.partial_ratio(s, wrong_str) > threshold for wrong_str in ['karaoke', 'reprise', 'solo', 'acoustic']]):
            # if the string has not been compared yet, compare it to all the other strings
            for j, other_strs in enumerate(str_list):
                if similar_strs[j] is None:
                    # if the other string has not been compared yet, compare them
                    if fuzz.token_sort_ratio(s, other_strs) > threshold:
                        # if the ratio is above 80, store the other string in the list
                        similar_strs[j] = s
    # if similar_strs still contains None values, replace them with the original string
    similar_strs = [s if s is not None else str_list[i] for i, s in enumerate(similar_strs)]
    return similar_strs

# find unique artists and replace duplicates
unique_artists = df["artist_old"].unique()
similar_artists = find_and_replace_duplicates(unique_artists)

# create a dataframe with the unique artists and the similar artists
df_artists = pd.DataFrame({"artist_old": unique_artists, "artist_new": similar_artists})

# replace artists in the original dataframe
df["artist"] = df["artist_old"].replace(df_artists.set_index("artist_old")["artist_new"])

# find unique titles and replace duplicates
for artist in df["artist_old"].unique():
    # select all tracks by the artist
    df_temp = df[df["artist_old"] == artist]

    # create list of similar titles
    unique_titles = df_temp["title_old"]
    similar_titles = find_and_replace_duplicates(unique_titles)

    # create a dataframe with the unique titles and the similar titles
    df_titles = pd.DataFrame({"title_old": unique_titles, "title_new": similar_titles}, index=df_temp.index)

    # replace values in the original dataframe at the specified index
    df.loc[df_titles.index, "title"] = df_titles["title_new"] 


In [ ]:
# export dataframe to csv
output_path = "../data/music_picks/model_statistics_deduplicated.csv"
df.to_csv(output_path, index=False)

In [162]:
# count unique combinations of artist and title for old and current data
print('Unique combinations of artist and title in the original data frame: {}'.format(df[['artist_old', 'title_old']].drop_duplicates().shape[0]))
print('Unique combinations of artist and title in the cleaned data frame: {}'.format(df[['artist', 'title']].drop_duplicates().shape[0]))

Unique combinations of artist and title in the original data frame: 4687
Unique combinations of artist and title in the cleaned data frame: 2733
